In [37]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OrdinalEncoder

from scipy.stats import chi2_contingency

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential 
from keras.layers import Dense
from keras.initializers import GlorotUniform
from keras.initializers import HeUniform
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adamax, Nadam, Ftrl

import hyperopt
from hyperopt import hp, fmin, tpe, Trials, space_eval, STATUS_OK

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

TEST_SIZE = 0.2


-------------------------


In [38]:
# Import del dataset e divisione in train e test
df = pd.read_csv('datasets/ObesityDataSet.csv')
# Sostituire la colonna 'NObeyesdad' con il risultato della seguente formula: Weight/(Height^2)
df['NObeyesdad'] = ((df['Weight']) / (df['Height']**2))
df.rename(columns={'NObeyesdad': 'label'}, inplace=True)

# Viene diviso il train set in train e validation set
train_df, test_df = train_test_split(df, test_size = TEST_SIZE, random_state = 42)
train_df, val_df = train_test_split(train_df, test_size = TEST_SIZE, random_state = 42)

train_x = train_df.drop(columns=['label'])
train_y = train_df['label'].astype('float64')

val_x = val_df.drop(columns=['label'])
val_y = val_df['label'].astype('float64')

test_x = test_df.drop(columns=['label'])
test_y = test_df['label'].astype('float64')

In [39]:
cat_cols = train_x.select_dtypes(include='object').columns
num_cols = train_x.select_dtypes(exclude='object').columns

scaler = StandardScaler()
train_x[num_cols] = scaler.fit_transform(train_x[num_cols])

# Applica l'encoding ai dati categoriali nella lista ord_list
encoder = OrdinalEncoder()
train_x[cat_cols] = encoder.fit_transform(train_x[cat_cols])

def fs(dataset, dataset_y):
    str_path = 'objects/features_obesity.npy'
    #controllare se esiste il file features.npy
    if not os.path.exists(str_path):
        dataset['label'] = dataset_y
        correlation_matrix=dataset.corr()
        features = correlation_matrix['label'][(correlation_matrix['label']>=0.1) | (correlation_matrix['label']<=-0.1)].index
        features = features.drop('label')
        print(correlation_matrix['label'].sort_values(ascending=False))
        print(features)
        np.save(str_path, features)
    features = np.load(str_path, allow_pickle=True)
    return train_x[features]

#se la colonna ha solo due valori diversi fare one hot encoding
def one_hot_encoding(dataset):
    for column in dataset.columns:
        if len(dataset[column].unique()) <= 2:
            dataset = pd.get_dummies(dataset, columns=[column])
    return dataset.astype('float64')

train_x = fs(train_x, train_y)
train_x = one_hot_encoding(train_x)
print(train_x.dtypes)


Age                                   float64
Height                                float64
Weight                                float64
FCVC                                  float64
CAEC                                  float64
CH2O                                  float64
FAF                                   float64
CALC                                  float64
family_history_with_overweight_0.0    float64
family_history_with_overweight_1.0    float64
FAVC_0.0                              float64
FAVC_1.0                              float64
SCC_0.0                               float64
SCC_1.0                               float64
dtype: object
